In [1]:
from label_studio_sdk.client import LabelStudio
from fishsense_api_sdk.client import Client
from pathlib import Path
import json

In [2]:
URL = "http://localhost:8000"

PROJECT_ID = 65

In [3]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [4]:
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [5]:
tasks = list(ls.tasks.list(project=PROJECT_ID))

len(tasks), tasks[0]

(11,
 LseTask(agreement=None, agreement_selected=None, annotations=[{'id': 130410, 'result': [{'id': '8b8SoqMe_u', 'type': 'keypointlabels', 'value': {'x': 22.005032334954162, 'y': 18.014766723479212, 'width': 0.12900288359386858, 'keypointlabels': ['Reference Point']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 3300, 'original_height': 2550}, {'id': 'e617B2qF3q', 'type': 'keypointlabels', 'value': {'x': 39.7865817987427, 'y': 18.11172628199491, 'width': 0.12900288359386858, 'keypointlabels': ['Reference Point']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 3300, 'original_height': 2550}, {'id': 'V40m1gnuja', 'type': 'keypointlabels', 'value': {'x': 58.62638600173624, 'y': 18.107487407389744, 'width': 0.12900288359386858, 'keypointlabels': ['Reference Point']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 3300, 'original

In [6]:
async with Client(URL) as client:
    dive_slates = await client.dive_slates.get()

len(dive_slates), dive_slates[0]

(11,
 DiveSlate(id=1, name='H-Slate', dpi=300, path='SMILE Calibration Slate.pdf', created_at=datetime.datetime(2025, 9, 6, 3, 53, 10, 189520, tzinfo=TzInfo(0)), reference_points=None))

In [7]:
name_to_task_map = {Path(task.data['img']).stem.split('-')[-1].replace('_', ' '): task for task in tasks}

name_to_task_map

{'Dive Slate 2': LseTask(agreement=None, agreement_selected=None, annotations=[{'id': 130410, 'result': [{'id': '8b8SoqMe_u', 'type': 'keypointlabels', 'value': {'x': 22.005032334954162, 'y': 18.014766723479212, 'width': 0.12900288359386858, 'keypointlabels': ['Reference Point']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 3300, 'original_height': 2550}, {'id': 'e617B2qF3q', 'type': 'keypointlabels', 'value': {'x': 39.7865817987427, 'y': 18.11172628199491, 'width': 0.12900288359386858, 'keypointlabels': ['Reference Point']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 3300, 'original_height': 2550}, {'id': 'V40m1gnuja', 'type': 'keypointlabels', 'value': {'x': 58.62638600173624, 'y': 18.107487407389744, 'width': 0.12900288359386858, 'keypointlabels': ['Reference Point']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 3300

In [9]:
async with Client(URL) as client:
    for dive_slate in dive_slates:
        task = name_to_task_map.get(dive_slate.path.split('.')[0].replace('#', ''))

        orignal_width = task.annotations[0]["result"][0]["original_width"]
        orignal_height = task.annotations[0]["result"][0]["original_height"]

        points = [(r["value"]["x"] / 100 * orignal_width, r["value"]["y"] / 100 * orignal_height) for r in task.annotations[0]["result"]]
        dive_slate.reference_points = points

        await client.dive_slates.put(dive_slate)
